In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [ ]:
! pip install language_tool_python

In [ ]:
! pip install better_profanity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 684.3 kB/s eta 0:00:00


In [ ]:
! pip install TextBlob

In [ ]:
from groq import Groq
import pandas as pd
import json
import time
import random
from tqdm import tqdm
import re
import nltk
import numpy as np
import torch
import nltk
from nltk.corpus import wordnet
import language_tool_python
from textblob import TextBlob
import re
from better_profanity import profanity
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
# Set your Groq API key
client = Groq(api_key='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')

### **1. Data Collection and Preparation**

- **Collect Data:**
    - [ ]  Generating a diverse set of survey questions using `llama3-8b-8192`

- **Annotate Data:**
    - [ ]  Label questions with their respective types.
                    1. Open-End
                    2. Multiple Choice
                    3. Rating-Scale
                    4. Yes/No
                    5. Likert-Scale
                    6. Spam
                    7. Phishing

- **Clean Data:**
    - [ ]  Standardize question formats.
    - [ ]  Normalize text

In [ ]:
TOPICS = [
    'customer satisfaction',
    'product feedback',
    'employee engagement',
    'market research',
    'user experience',
    'brand perception',
    'service quality',
    'Global Economy',
    'Politics',
    'Latest Technological Advancements'
]

QUESTION_TYPES = [
    'open-ended',
    'multiple-choice',
    'rating-scale',
    'yes-no',
    'likert-scale'
]

def generate_question(question_type, topic):
    prompt = f"""Generate a high-quality {question_type} question about {topic}.
    Provide only the question itself, without any introductory text or numbering.
    Ensure the question is clear, concise, and relevant to the topic.
    For multiple-choice questions, include 4 options and also provide options in bulletins and also make sure it won't resemble rating-scale, likert-scale or yes/no questions.
    For rating-scale questions, use a 1-5 scale. Be mindful to the context. It has to be clearly a rating-scale question.
    For Likert-scale questions, use a 5-point scale from Strongly Disagree to Strongly Agree. Be mindful to the context. It has to be clearly a Likert-scale question.

    Question:"""

    try:
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are a survey design expert. Provide only the requested question without any additional text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=100,
        )

        # Extract and clean the question
        raw_response = completion.choices[0].message.content.strip()

        # Remove any numbering or bullet points
        cleaned_question = re.sub(r'^\d+\.?\s*|\*\s*', '', raw_response)

        # Remove any "Question:" prefix if present
        cleaned_question = re.sub(r'^Question:\s*', '', cleaned_question)

        # Ensure the question ends with a question mark
        if not cleaned_question.endswith('?'):
            cleaned_question += '?'

        return cleaned_question.strip()
    except Exception as e:
        print(f"Error generating question: {e}")
        return None

def generate_spam_question():
    spam_words = [
        "skype", "sky***pe", "hotmail", "h-o-t-m-a-i-l", "hotm***l", "outlook", "outl**k",
        "o**ut**look", "login", "verify", "verifxx", "signup", "sign in to", "e-m-a-i-l",
        "log**in", "ver**y", "v***fy", "gmail", "gma**il", "yahoo", "ya*oo", "y*h*oo",
        "email", "bt", "validate", "click", "clickhere", "signin"
    ]
    prompt = f"""Generate a spam question that includes at least one of the following words or phrases:
    {', '.join(spam_words)}
    Make the question sound enticing but clearly spam-like. Provide only the question itself, without any introductory text or numbering.

    Question:"""

    try:
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are generating spam questions. Make them enticing but clearly spam-like. Provide only the question itself."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=100,
        )

        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating spam question: {e}")
        return None

def generate_phishing_question():
    phishing_words = [
        "credentials", "credential", "password", "pass word", "p a s s w o r d", "p.a.s.s.w.o.r.d",
        "fjalëkalim", "pasahitza", "пароль", "lozinka", "парола", "contrasenya", "lozinka", "heslo",
        "adgangskode", "wachtwoord", "parool", "salasana", "mot de passe", "contrasinal", "passwort",
        "kωδικός πρόσβασης", "jelszó", "lykilorð", "focal faire", "parola d'ordine", "parole",
        "slaptažodis", "лозинка", "passord", "hasło", "senha", "parola", "пароль", "лозинка", "heslo",
        "geslo", "contraseña", "Lösenord", "пароль", "cyfrinair", "פּאַראָל", "գաղտնաբառ", "parol",
        "পাসওয়ার্ড", "დაგავიწყდათ", "પાસવર્ડ", "पासवर्ड", "lo lus zais", "パスワード", "ಪಾಸ್ವರ್ಡ್",
        "пароль", "ការពាក្យសម្ងាត់", "암호", "ລະຫັດຜ່ານ", "पासवर्ड", "нууц үг", "စကားဝှက်ကို", "पासवर्ड",
        "මුරපදය", "гузарвожа", "கடவுச்சொல்", "పాస్వర్డ్", "รหัสผ่าน", "پاس ورڈ", "parol", "mật khẩu",
        "كلمه السر", "סיסמה", "کلمه عبور", "parola", "wagwoord", "achinsinsi", "kalmar sirri",
        "paswọọdụ", "phasewete", "ọrọigbaniwọle", "iphasiwedi", "kata sandi", "sandi", "tenimiafina",
        "kata laluan", "kupuhipa", "pasvorto", "creole modpas", "p***word", "pa****ssword", "p*******d",
        "p******d", "pasw0rd", "passw0rd", "pasword", "passwcrd"
    ]
    prompt = f"""Generate a phishing question that includes at least one of the following words or phrases:
    {', '.join(phishing_words)}
    Make the question sound urgent and official, but clearly phishing-like. Provide only the question itself, without any introductory text or numbering.

    Question:"""

    try:
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are generating phishing questions. Make them sound urgent and official, but clearly phishing-like. Provide only the question itself."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=100,
        )

        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating phishing question: {e}")
        return None

def generate_questions_for_type(question_type, num_questions_per_topic=10):
    questions = []

    for topic in tqdm(TOPICS, desc=f"Generating {question_type} questions"):
        for _ in range(num_questions_per_topic):
            question = generate_question(question_type, topic)

            if question:
                questions.append({
                    'question': question,
                    'type': question_type,
                    'topic': topic,
                    'spam': False,
                    'phishing': False
                })

            time.sleep(1)  # To avoid hitting API rate limits

    return questions

# Main execution
all_questions = []

for question_type in QUESTION_TYPES:
    questions = generate_questions_for_type(question_type, num_questions_per_topic=10)
    all_questions.extend(questions)

# Generate spam and phishing questions
num_spam_phishing = 100  # Adjust as needed

for _ in tqdm(range(num_spam_phishing), desc="Generating spam questions"):
    spam_question = generate_spam_question()
    if spam_question:
        all_questions.append({
            'question': spam_question,
            'type': 'spam',
            'topic': 'N/A',
            'spam': True,
            'phishing': False
        })
    time.sleep(1)

for _ in tqdm(range(num_spam_phishing), desc="Generating phishing questions"):
    phishing_question = generate_phishing_question()
    if phishing_question:
        all_questions.append({
            'question': phishing_question,
            'type': 'phishing',
            'topic': 'N/A',
            'spam': False,
            'phishing': True
        })
    time.sleep(1)

# Save the dataset as CSV
df = pd.DataFrame(all_questions)
df.to_csv('survey_questions_dataset.csv', index=False)

print("\nDataset generated and saved as 'survey_questions_dataset.csv'")

Generating phishing questions: 100%|██████████| 100/100 [04:17<00:00,  2.58s/it]


Dataset generated and saved as 'survey_questions_dataset.csv'


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SmartSurveyBuilder/survey_questions_dataset.csv')
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,question,type,topic,spam,phishing
0,"""URGENT: Your online account credentials have ...",phishing,NaN,False,True
1,Did the service representative resolve your is...,yes-no,service quality,False,False
2,What do you believe is the most critical issue...,open-ended,Politics,False,False
3,Can't access your hotmail account after trying...,spam,NaN,True,False
4,What impact do you think the latest technologi...,open-ended,Latest Technological Advancements,False,False


### **2. Feature Engineering**

- **Tokenize Text:**
    - [ ]  Break down questions into tokens.
- **Tag Parts of Speech:**
    - [ ]  Identify grammatical parts of speech for each token.
- **Recognize Named Entities:**
    - [ ]  Extract entities such as names, locations, and dates.
- **Vectorize Text:**
    - [ ]  Convert text into numerical vectors using Bag of Words, TF-IDF

In [ ]:
# Tokenization

nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize_question(question):
    return word_tokenize(question)

# Apply tokenization to the 'question' column
df['tokens'] = df['question'].apply(tokenize_question)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Parts of Speech Tags

nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag

def pos_tag_question(tokens):
    return pos_tag(tokens)

df['pos_tags'] = df['tokens'].apply(pos_tag_question)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Named Entity Recognition

nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk

def extract_entities(pos_tags):
    return ne_chunk(pos_tags)

df['named_entities'] = df['pos_tags'].apply(extract_entities)

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
# Text Vectorization

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['question'])

# Convert to DataFrame for easier viewing
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

tfidf_df.head()

,000,10,1000,12,19,1gb,24,30,50,500,...,worldwide,worth,would,xyz,ya,yahoo,year,years,you,your
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.190510
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.129789
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.207359,0.098905
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.252802
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.21872,0.090311,0.000000


### **3. Model Selection and Training - Question Classification**

-  **Data Augmentation:**
    - [ ]  Augmented the data using Synonyms replacements for additional training samples to enhance model's robustness
-  **Select Model:**
    - [ ]  RoBERTa, an optimized version of BERT is used for question classification task.
- **Model Training with Customization:**
    - [ ]  Implemented a cosine learning rate schedule with warmup, which often leads to better convergence.
    - [ ]  Unfroze the last 4 layers of RoBERTa for fine-tuning, allowing the model to adapt more to our specific task.
    - [ ] Implemented early stopping to prevent overfitting.
    - [ ] Added gradient clipping to prevent exploding gradients.
- **Validate Models:**
    - [ ]  Regular validation of performance with a separate validation set.

In [ ]:
nltk.download('wordnet')

# Encode labels
le = LabelEncoder()

labels = le.fit_transform(df['type'])
if isinstance(labels, np.ndarray):
    labels = labels.tolist()
questions = df['question'].tolist()

# 1. Data Preparation and Augmentation
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word.isalnum()]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return new_words

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def augment_data(questions, labels):
    augmented_questions = []
    augmented_labels = []
    for question, label in zip(questions, labels):
        augmented_questions.append(question)
        augmented_labels.append(label)
        words = question.split()
        augmented_words = synonym_replacement(words, n=int(len(words) * 0.1))
        augmented_questions.append(' '.join(augmented_words))
        augmented_labels.append(label)
    return augmented_questions, augmented_labels

augmented_questions, augmented_labels = augment_data(questions, labels)

questions.extend(augmented_questions)
labels.extend(augmented_labels)

X_train, X_val, y_train, y_val = train_test_split(questions, labels, test_size=0.2, random_state=42)

# 2. Model and Tokenizer Initialization
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(le.classes_))

for param in model.roberta.encoder.layer[-4:].parameters():
    param.requires_grad = True

# 3. Data Encoding and DataLoader Creation
def encode_questions(questions):
    return tokenizer(questions, padding=True, truncation=True, max_length=128, return_tensors="pt")

train_encodings = encode_questions(X_train)
val_encodings = encode_questions(X_val)

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(y_train))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val))

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 4. Training Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 10  # Increased number of epochs
total_steps = len(train_loader) * num_epochs
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=total_steps//10, num_training_steps=total_steps)

# 5. Training and Evaluation Loop
best_accuracy = 0
patience = 3
no_improve = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Average training loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_accuracy = 0
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            val_accuracy += (predictions == labels).float().mean()
            val_predictions.extend(predictions.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    val_accuracy /= len(val_loader)
    print(f"Validation Accuracy: {val_accuracy:.4f}")

    print("\nClassification Report:")
    print(classification_report(val_true_labels, val_predictions, target_names=le.classes_))

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_roberta_model.pth')
        no_improve = 0
    else:
        no_improve += 1

    if no_improve == patience:
        print("Early stopping triggered")
        break

# 6. Load Best Model
model.load_state_dict(torch.load('best_roberta_model.pth'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epoch 9/10: 100%|██████████| 105/105 [01:24<00:00,  1.25it/s]


Average training loss: 0.0334


Validation: 100%|██████████| 27/27 [00:05<00:00,  5.05it/s]


Validation Accuracy: 0.9699

Classification Report:
                 precision    recall  f1-score   support

   likert-scale       0.96      0.95      0.96        58
multiple-choice       0.97      0.94      0.95        67
     open-ended       1.00      1.00      1.00        58
       phishing       1.00      1.00      1.00        59
   rating-scale       0.91      0.95      0.93        64
           spam       1.00      1.00      1.00        66
         yes-no       1.00      1.00      1.00        48

       accuracy                           0.98       420
      macro avg       0.98      0.98      0.98       420
   weighted avg       0.98      0.98      0.98       420

Early stopping triggered


<All keys matched successfully>

In [ ]:
# 7. Function to Classify New Questions

def classify_question(question):
    model.eval()
    encoding = tokenizer(question, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=-1)
    return le.inverse_transform(prediction.cpu().numpy())[0]

# 8. Example Usage
new_question = "Please provide your email-id and password."
predicted_type = classify_question(new_question)
print(f"Type Prediction: {predicted_type}")

Type Prediction: phishing


### **4. Quality Assurance**

- [ ]  **Check Grammar:**
    - [ ]  Used the language_tool_python library to check for grammatical errors.
- [ ]  **Analyze Tone:**
    - [ ]   Used TextBlob to analyze the sentiment/tone of the question.
- [ ]  **Detect Profanity:**
    - [ ]   Used the better_profanity library to detect any profane words.
- [ ]  **Identify Low-Quality Text:**
    - [ ]   Implements simple heuristics such as being very short, all caps and repetitive characters to identify potentially low-quality questions.

In [ ]:
# Initialize grammar checker
grammar_tool = language_tool_python.LanguageTool('en-US')

# 1. Grammar Check
def check_grammar(question):
    matches = grammar_tool.check(question)
    if matches:
        return False, [match.message for match in matches]
    return True, []

# 2. Tone Analysis
def analyze_tone(question):
    blob = TextBlob(question)
    polarity = blob.sentiment.polarity
    if polarity < -0.5:
        return "Very Negative"
    elif -0.5 <= polarity < -0.1:
        return "Negative"
    elif -0.1 <= polarity <= 0.1:
        return "Neutral"
    elif 0.1 < polarity <= 0.5:
        return "Positive"
    else:
        return "Very Positive"

# 3. Profanity Check
profanity.load_censor_words()

def check_profanity(question):
    return not profanity.contains_profanity(question)

# 4. Low-Quality Text Detection
def is_low_quality(question):
    # Check for very short questions
    if len(question.split()) < 3:
        return True

    # Check for all caps
    if question.isupper():
        return True

    # Check for repetitive characters
    if re.search(r'(.)\1{2,}', question):
        return True

    return False

INFO:language_tool_python.download_lt:Unzipping /tmp/tmp52__f6vo.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


---

In [ ]:
# Comprehensive Question Check

def overall_check(questions):
    questions_overall_results = []
    results = {
        "original_question": None,
        "predicted_type": None,
        "grammar_check": None,
        "grammar_issues": None,
        "tone": None,
        "profanity_free": None,
        "quality_text": None,
        "spam": False,
        "phishing": False,
        "overall_pass": False
    }

    for question in questions:
        results["original_question"] = question

        # Type prediction
        results["predicted_type"] = classify_question(question)

        # Grammar check
        results["grammar_check"], results["grammar_issues"] = check_grammar(question)

        # Tone analysis
        results["tone"] = analyze_tone(question)

        # Profanity check
        results["profanity_free"] = check_profanity(question)

        # Low-quality text check
        results["quality_text"] = not is_low_quality(question)

        # Spam Check
        if results["predicted_type"] == 'spam':
            results['spam'] = True
        else:
            results['spam'] = False

        # Phishing Check
        if results["predicted_type"] == 'phishing':
            results['phishing'] = True
        else:
            results['phishing'] = False

        # Overall pass/fail
        results["overall_pass"] = (
            results["grammar_check"] and
            results["profanity_free"] and
            results["quality_text"] and
            results["spam"] is False and
            results["phishing"] is False
        )

        questions_overall_results.append(results)

    return questions_overall_results

##  Driver Code

Users can give their questions here to predict the question type and check the quality of the question

In [ ]:
sample_questions = [
    "How do you think artificial intelligence will impact the job market in the next decade?",
    "Which aspect of customer service do you value most? A) Speed of response, B) Friendliness of staff, C) Problem-solving ability, D) Follow-up communication",
    "On a scale of 1-5, how satisfied are you with the current state of public transportation in your city?",
    "Do you believe that renewable energy sources will completely replace fossil fuels within the next 50 years?",
    "I feel that my company values my contributions to the team. (Strongly Disagree) 1 2 3 4 5 (Strongly Agree)",
    "Congratulations! You've been selected to receive a free iPad! Click here to claim your prize now!",
    "Your account has been temporarily suspended due to unusual activity. Please verify your password to regain access.",
    "What strategies do you use to maintain a healthy work-life balance?",
    "Which global issue concerns you the most? A) Climate change, B) Economic inequality, C) Political instability, D) Pandemic preparedness",
    "On a scale of 1-5, how likely are you to recommend our product to a friend or colleague?",
    "Have you ever participated in a virtual reality experience?",
    "Social media has a positive impact on society. (Strongly Disagree) 1 2 3 4 5 (Strongly Agree)",
    "Want to make $5000 a week working from home? Learn this one weird trick!",
    "We've detected a security breach on your account. Please enter your login credentials to verify your identity and secure your account.",
    "How has the rise of e-commerce affected your shopping habits?"
]

results = overall_check(sample_questions)

for result in results:
    print(f"Question: {result['original_question']}")
    print(f"Predicted Type: {result['predicted_type']}")
    print(f"Grammar Check: {'Pass' if result['grammar_check'] else 'Fail'}")
    if not result['grammar_check']:
        print(f"Grammar Issues: {result['grammar_issues']}")
    print(f"Tone: {result['tone']}")
    print(f"Profanity Free: {'Yes' if result['profanity_free'] else 'No'}")
    print(f"Quality Text: {'Yes' if result['quality_text'] else 'No'}")
    print(f"Spam: {'Yes' if result['spam'] else 'No'}")
    print(f"Phishing: {'Yes' if result['phishing'] else 'No'}")
    print(f"Overall Pass: {'Yes' if result['overall_pass'] else 'No'}")
    print("---")

Question: How do you think artificial intelligence will impact the job market in the next decade?
Predicted Type: open-ended
Grammar Check: Pass
Tone: Negative
Profanity Free: Yes
Quality Text: Yes
Spam: No
Phishing: No
Overall Pass: Yes
---
Question: Which aspect of customer service do you value most? A) Speed of response, B) Friendliness of staff, C) Problem-solving ability, D) Follow-up communication
Predicted Type: multiple-choice
Grammar Check: Fail
Grammar Issues: ['Unpaired symbol: ‘(’ seems to be missing', 'Unpaired symbol: ‘(’ seems to be missing']
Tone: Positive
Profanity Free: Yes
Quality Text: Yes
Spam: No
Phishing: No
Overall Pass: No
---
Question: On a scale of 1-5, how satisfied are you with the current state of public transportation in your city?
Predicted Type: rating-scale
Grammar Check: Pass
Tone: Positive
Profanity Free: Yes
Quality Text: Yes
Spam: No
Phishing: No
Overall Pass: Yes
---
Question: Do you believe that renewable energy sources will completely replace fo

### **5. Automated Question Generation**

- [ ]  **Generate via Templates:**
    - [ ]  Use predefined templates to create questions based on user input.
- [ ]  **Integrate LLMs:**
    - [ ]  Leverage LLMs to generate context-aware questions.
- [ ]  **Apply Quality Checks:**
    - [ ]  Ensure generated questions meet quality standards using QA steps.

In [ ]:
# Initialize Groq client
client = Groq(api_key='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')

# Initialize grammar checker
grammar_tool = language_tool_python.LanguageTool('en-US')

# Load profanity words
profanity.load_censor_words()

# Custom templates
TEMPLATES = {
    "open_ended": [
        "What are your thoughts on {topic} in relation to {aspect}?",
        "How does {topic} impact your daily life, especially regarding {aspect}?",
        "Can you describe a situation where {topic} significantly affected {aspect}?",
        "What improvements would you suggest for {topic} to enhance {aspect}?",
        "How do you think {topic} will evolve in the next 5 years, particularly in terms of {aspect}?"
    ],
    "mcq": [
        "Which of the following best describes your experience with {topic} in terms of {aspect}?\n"
        "A) Very Positive\nB) Somewhat Positive\nC) Neutral\nD) Somewhat Negative\nE) Very Negative",

        "What do you consider the most important factor in {topic} when it comes to {aspect}?\n"
        "A) Cost\nB) Quality\nC) Innovation\nD) Accessibility\nE) Environmental Impact",

        "How often do you engage with {topic}, particularly in relation to {aspect}?\n"
        "A) Daily\nB) Weekly\nC) Monthly\nD) Rarely\nE) Never"
    ],
    "rating": [
        "On a scale of 1 to 5, with 1 being very dissatisfied and 5 being very satisfied, "
        "how would you rate your experience with {topic} in terms of {aspect}?",

        "From 1 to 10, with 1 being not at all important and 10 being extremely important, "
        "how would you rate the significance of {aspect} in relation to {topic}?"
    ],
    "likert": [
        "Please indicate your level of agreement with the following statement:\n"
        "'{topic} has a positive impact on {aspect}.'\n"
        "Strongly Disagree / Disagree / Neutral / Agree / Strongly Agree",

        "To what extent do you agree that {topic} is essential for improving {aspect}?\n"
        "Strongly Disagree / Disagree / Neutral / Agree / Strongly Agree"
    ],
    "yes_no": [
        "Have you ever experienced a significant improvement in {aspect} due to {topic}? (Yes/No)",
        "Do you believe that {topic} will continue to play a crucial role in {aspect} in the future? (Yes/No)"
    ]
}

ASPECTS = [
    "efficiency", "user experience", "cost-effectiveness", "sustainability",
    "innovation", "accessibility", "reliability", "social impact",
    "ethical considerations", "future potential"
]

def generate_question_from_template(topic):
    question_type = random.choice(list(TEMPLATES.keys()))
    template = random.choice(TEMPLATES[question_type])
    aspect = random.choice(ASPECTS)
    return template.format(topic=topic, aspect=aspect)


def generate_and_check_question(topic):
    question = generate_question_from_template(topic)
    qa_results = quality_check(question)

    if qa_results['overall_pass']:
        return question, qa_results
    else:
        return generate_and_check_question(topic)

def generate_high_quality_questions(topic, num_questions):
    all_questions = []
    quality_summary = {
        "total_questions": num_questions,
        "passed_questions": 0,
        "failed_questions": 0,
        "grammar_issues": 0,
        "profanity_issues": 0,
        "quality_issues": 0
    }

    for _ in tqdm(range(num_questions), desc=f"Generating questions about {topic}"):
        question, qa_results = generate_and_check_question(topic)
        all_questions.append({
            'question': question,
            'grammar_check': qa_results['grammar_check'],
            'tone': qa_results['tone'],
            'profanity_free': qa_results['profanity_free'],
            'quality_text': qa_results['quality_text']
        })

        if qa_results['overall_pass']:
            quality_summary["passed_questions"] += 1
        else:
            quality_summary["failed_questions"] += 1

        if not qa_results['grammar_check']:
            quality_summary["grammar_issues"] += 1
        if not qa_results['profanity_free']:
            quality_summary["profanity_issues"] += 1
        if not qa_results['quality_text']:
            quality_summary["quality_issues"] += 1

    return all_questions, quality_summary

def print_quality_summary(summary):
    print("\nQuality Check Summary:")
    print(f"Total questions generated: {summary['total_questions']}")
    print(f"Questions that passed all checks: {summary['passed_questions']}")
    print(f"Questions that failed one or more checks: {summary['failed_questions']}")
    print(f"Questions with grammar issues: {summary['grammar_issues']}")
    print(f"Questions with profanity: {summary['profanity_issues']}")
    print(f"Questions with quality issues: {summary['quality_issues']}")

# Main execution
if __name__ == "__main__":
    topic = input("Enter your preferred topic of interest: ")
    num_questions = int(input("Enter the number of questions you want to generate: "))

    questions, quality_summary = generate_high_quality_questions(topic, num_questions)

    # Print quality summary
    print_quality_summary(quality_summary)

    # Save the questions as CSV
    df = pd.DataFrame(questions)
    filename = f"{topic.replace(' ', '_')}_survey_questions.csv"
    df.to_csv(filename, index=False)

    print(f"\nQuestions generated and saved as '{filename}'")

    # Display the first 5 questions (or all if less than 5)
    print("\nSample questions:")
    for i, q in enumerate(questions[:5], 1):
        print(f"{i}. {q['question']}")

    if len(questions) > 5:
        print("...")

Enter your preferred topic of interest: Customer Satisfaction
Enter the number of questions you want to generate: 10


Generating questions about Customer Satisfaction: 100%|██████████| 10/10 [00:04<00:00,  2.31it/s]



Quality Check Summary:
Total questions generated: 10
Questions that passed all checks: 10
Questions that failed one or more checks: 0
Questions with grammar issues: 0
Questions with profanity: 0
Questions with quality issues: 0

Questions generated and saved as 'Customer_Satisfaction_survey_questions.csv'

Sample questions:
1. Do you believe that Customer Satisfaction will continue to play a crucial role in ethical considerations in the future? (Yes/No)
2. Please indicate your level of agreement with the following statement:
'Customer Satisfaction has a positive impact on cost-effectiveness.'
Strongly Disagree / Disagree / Neutral / Agree / Strongly Agree
3. Please indicate your level of agreement with the following statement:
'Customer Satisfaction has a positive impact on future potential.'
Strongly Disagree / Disagree / Neutral / Agree / Strongly Agree
4. What do you consider the most important factor in Customer Satisfaction when it comes to social impact?
A) Cost
B) Quality
C) In